In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df = pd.read_csv('Telco-Customer-Churn Dataset.csv')

In [5]:
df.head(15)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,No,Mailed check,29.75,301.9,No
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,No,No,No,One year,No,Bank transfer (automatic),56.15,3487.95,No


In [6]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [7]:
def produce_missing_report(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_value_df = pd.DataFrame({'column_name': df.columns,
                                     'percent_missing': percent_missing})
    return missing_value_df

In [8]:
produce_missing_report(df)

,column_name,percent_missing
customerID,customerID,0.0
gender,gender,0.0
SeniorCitizen,SeniorCitizen,0.0
Partner,Partner,0.0
Dependents,Dependents,0.0
tenure,tenure,0.0
PhoneService,PhoneService,0.0
MultipleLines,MultipleLines,0.0
InternetService,InternetService,0.0
OnlineSecurity,OnlineSecurity,0.0


In [11]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [13]:
df[df.duplicated()]

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn


In [15]:
to_drop = ['customerID', 'Partner', 'Dependents', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling']
df = df.drop(to_drop, axis=1)

In [17]:
df.dtypes

gender              object
SeniorCitizen        int64
tenure               int64
PhoneService        object
MultipleLines       object
InternetService     object
Contract            object
PaymentMethod       object
MonthlyCharges     float64
TotalCharges        object
Churn               object
dtype: object

In [34]:
ohe = OneHotEncoder()
le = LabelEncoder()

Gender: Male-0, Female-1

In [39]:
df['gender'] = le.fit_transform(df['gender'])

PhoneService: Yes-1, No-0

In [40]:
df['PhoneService'] = le.fit_transform(df['PhoneService'])

MultipleLines: No-0, No PhoneService-1, Yes-2

In [41]:
df['MultipleLines'] = le.fit_transform(df['MultipleLines'])

InternetService: DSL-0, Fiber Optic-1, No-2

In [42]:
df['InternetService'] = le.fit_transform(df['InternetService'])

Contract: Month to Month-0, 1 Year-1, 2 Years-2

In [43]:
df['Contract'] = le.fit_transform(df['Contract'])

PaymentMethod: Bank Transfer-0, Credit Card-1, Electronic Check-2, Mailed Check-3

In [44]:
df['PaymentMethod'] = le.fit_transform(df['PaymentMethod'])

Churn: No-0, Yes-1

In [51]:
df['Churn'] = le.fit_transform(df['Churn'])